In [3]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('figure', figsize = (10,6))
np.set_printoptions(precision = 4, suppress = True)

In [ ]:
# Import two datasets
colnames = ["LOAN_ID", "ORIG_CHN", "Seller.Name", "ORIG_RT", "ORIG_AMT", "ORIG_TRM", "ORIG_DTE","FRST_DTE", "OLTV", 
            "OCLTV", "NUM_BO", "DTI", "CSCORE_B", "FTHB_FLG", "PURPOSE", "PROP_TYP","NUM_UNIT", "OCC_STAT", 
            "STATE", "ZIP_3", "MI_PCT", "Product.Type", "CSCORE_C", "MI_TYPE", "RELOCATION_FLG"]
Data_A = pd.read_table('D:/1-Intern/Project_1/2018Q1/Acquisition_2018Q1.txt', 
                       sep = '|', header = None, names = colnames)
colnames_P = ["LOAN_ID", "Monthly.Rpt.Prd", "Servicer.Name", "LAST_RT", "LAST_UPB", "Loan.Age", "Months.To.Legal.Mat", 
              "Adj.Month.To.Mat", "Maturity.Date", "MSA", "Delq.Status", "MOD_FLAG", "Zero.Bal.Code", 
              "ZB_DTE", "LPI_DTE", "FCC_DTE","DISP_DT", "FCC_COST", "PP_COST", "AR_COST", "IE_COST", "TAX_COST", "NS_PROCS",
              "CE_PROCS", "RMW_PROCS", "O_PROCS", "NON_INT_UPB", "PRIN_FORG_UPB_FHFA", "REPCH_FLAG", "PRIN_FORG_UPB_OTH", "TRANSFER_FLG"]
Data_P = pd.read_table('D:/1-Intern/Project_1/2018Q1/Performance_2018Q1.txt', 
                       sep = '|', header = None, names = colnames_P,low_memory=False)

In [ ]:
df = pd.merge(Data_A.loc[:,["LOAN_ID", "ORIG_CHN","ORIG_RT", "ORIG_AMT", "ORIG_TRM", "OLTV", "DTI", "CSCORE_B", "FTHB_FLG","STATE" ]],
              Data_P.loc[:,["LOAN_ID", "Monthly.Rpt.Prd", "LAST_RT", "LAST_UPB","Delq.Status", "MOD_FLAG", "Zero.Bal.Code", "ZB_DTE", "FCC_DTE"]] , on = 'LOAN_ID')

In [116]:
df.shape

(426206, 24)

In [11]:
# Convert string into datetime
from datetime import datetime
df['Monthly.Rpt.Prd'] = [datetime.strptime(x, '%m/%d/%Y') for x in df.loc[:,'Monthly.Rpt.Prd']]

In [12]:
df.dtypes

LOAN_ID                     int64
ORIG_RT                   float64
ORIG_AMT                    int64
ORIG_TRM                    int64
OLTV                        int64
DTI                       float64
CSCORE_B                  float64
FTHB_FLG                   object
STATE                      object
Monthly.Rpt.Prd    datetime64[ns]
LAST_RT                   float64
LAST_UPB                  float64
Delq.Status                object
MOD_FLAG                   object
Zero.Bal.Code             float64
ZB_DTE                     object
FCC_DTE                    object
dtype: object

In [16]:
# coding Delinquncy status into 0,1,2,3,4+ categories
df['Delq.Status_Coding'] = df['Delq.Status'].replace(['4','5','6','7','8','9','10','11','12','13',
                                                      '14','15','16','17','18','19','20','21','22'], '4+')
# Understand how many categories in Delinquency status
df['Delq.Status_Coding'].value_counts(dropna=False)

0      9187791
X        68114
1        52514
2         9927
4+        9424
3         4143
NaN        137
Name: Delq.Status_Coding, dtype: int64

In [15]:
df['Delq.Status'].value_counts(dropna=False)

0      9187791
X        68114
1        52514
2         9927
3         4143
4         2669
5         2016
6         1336
7          941
8          648
9          446
10         375
11         274
12         199
13         158
NaN        137
14         119
15          96
16          51
17          38
18          24
19          16
20          11
21           5
22           2
Name: Delq.Status, dtype: int64

In [17]:
df['Zero.Bal.Code'].value_counts(dropna=False)

NaN     9263800
1.0       67775
6.0         326
9.0          89
2.0          31
16.0         12
3.0          12
15.0          5
Name: Zero.Bal.Code, dtype: int64

In [18]:
#From the below result, we coukd know that only when delinquency status is unknown, Zero Balance code is denoted
df.groupby(['Delq.Status_Coding','Zero.Bal.Code']).size()

Delq.Status_Coding  Zero.Bal.Code
X                   1.0              67775
                    6.0                326
                    16.0                12
dtype: int64

In [19]:
# coding FCC as 'F'
df['FCC_DTE_Coding'] = df['FCC_DTE'].replace(['10/01/2019', '05/01/2019', '08/01/2019', '12/01/2018',
       '04/01/2019', '07/01/2019', '01/01/2019', '09/01/2019',
       '06/01/2019', '12/01/2019', '11/01/2018', '03/01/2019',
       '02/01/2019', '11/01/2019', '10/01/2018', '07/01/2018'],'F')

In [20]:
df['FCC_DTE_Coding'].value_counts(dropna=False)

NaN    9331924
F          126
Name: FCC_DTE_Coding, dtype: int64

In [21]:
# From below result, we know that Foreclosure only shows when loans denoted as 2.0, 3.0, 9.0. 

df.groupby(['FCC_DTE_Coding','Zero.Bal.Code']).size()

FCC_DTE_Coding  Zero.Bal.Code
F               2.0              31
                3.0               9
                9.0              86
dtype: int64

In [22]:
# In addition, when 'Delq.Status_Coding' shows, Foreclosure would not be come up. they totally exlusive. Combined with result above, 
# we could just denote 2.0, 3.0, 9.0. as 'F'
df.groupby(['FCC_DTE_Coding','Delq.Status_Coding']).size()

Series([], dtype: int64)

In [23]:
df['Zero.Bal.Code']=df['Zero.Bal.Code'].replace([2.0, 3.0, 9.0],'F')

In [24]:
df['Zero.Bal.Code'].value_counts(dropna=False)

NaN     9263800
1.0       67775
6.0         326
F           132
16.0         12
15.0          5
Name: Zero.Bal.Code, dtype: int64

In [25]:
df['Delq.Status_Coding'].value_counts()

0     9187791
X       68114
1       52514
2        9927
4+       9424
3        4143
Name: Delq.Status_Coding, dtype: int64

In [26]:
conditions = [
    df['Delq.Status_Coding']== '0' ,
    df['Delq.Status_Coding']== '1',
    df['Delq.Status_Coding']== '2',
    df['Delq.Status_Coding']== '3',
    df['Delq.Status_Coding']== '4+',
    ((pd.isna(df['Delq.Status_Coding']))& (df['Zero.Bal.Code']== 'F' )),
    ((df['Delq.Status_Coding']=='X' )& (df['Zero.Bal.Code']== 1.0 )),
    ((df['Delq.Status_Coding']=='X' )& (df['Zero.Bal.Code']== 6.0 )),
    ((df['Delq.Status_Coding']=='X' )& (df['Zero.Bal.Code']== 16.0 )),
    ((df['Delq.Status_Coding']=='X' )& (pd.isna(df['Zero.Bal.Code']))),
    ((pd.isna(df['Delq.Status_Coding']))& (df['Zero.Bal.Code']== 15.0))
]

In [27]:
choises = ['0','1','2','3','4+','F','P','Other','Other','Other','Other']

In [28]:
df['Loan_Status'] = np.select(condlist=conditions, choicelist = choises)

In [19]:
df['Loan_Status'].value_counts()

0        9187791
P          67775
1          52514
2           9927
4+          9424
3           4143
Other        344
F            132
Name: Loan_Status, dtype: int64

In [29]:
df = df.pivot('LOAN_ID','Monthly.Rpt.Prd','Loan_Status')

In [23]:
df.head

<bound method NDFrame.head of Monthly.Rpt.Prd 2018-01-01 2018-02-01 2018-03-01 2018-04-01 2018-05-01  \
LOAN_ID                                                                  
100001040173           NaN          0          0          0          0   
100002370993           NaN        NaN          0          0          0   
100005405807             0          0          0          0          0   
100008071646           NaN        NaN          0          0          0   
100010739040           NaN          0          0          0          0   
...                    ...        ...        ...        ...        ...   
999991303485             0          0          0          0          0   
999992661330           NaN          0          0          0          0   
999995056050           NaN          0          0          0          0   
999997685206           NaN        NaN          0          0          0   
999998783494           NaN          0          0          0          0   

Monthly

In [30]:
table_1 = pd.crosstab(df['2018-01-01'],df['2018-02-01'], margins=True)
table_1  

2018-02-01 00:00:00,0,1,2,Other,P,All
2018-01-01 00:00:00,,,,,,
0,153320,903,1,1,225,154450
1,84,7,9,0,0,100
All,293478,983,10,1,225,426206


In [59]:
table_1_P = pd.crosstab(df['2018-01-01'],df['2018-02-01'], normalize='index',margins=True)
table_1_P.style.background_gradient(cmap='Blues',axis=None) 


2018-02-01 00:00:00,0,1,2,Other,P
2018-01-01 00:00:00,,,,,
0,0.992684,0.005847,0.000006,0.000006,0.001457
1,0.840000,0.070000,0.090000,0.000000,0.000000
All,0.992585,0.005888,0.000065,0.000006,0.001456
